In [3]:
pip install ipeadatapy


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
from bcb import sgs
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import ipeadatapy as ip

warnings.simplefilter("ignore")

In [9]:
ipca = sgs.get(('IPCA', 433))
cambio = sgs.get(('Câmbio', 1))
pib = sgs.get(('PIB', 4380))
igpm = sgs.get(('IGPM', 189))
igpdi = sgs.get(('IGPDI', 190))
M1 = sgs.get(('M1', 27815))
empregos = sgs.get(('empregos', 28763))
bp = sgs.get(('bp', 22707))
divida = sgs.get(('divida', 4003))
ipca_esperado = pd.read_csv("C:\\Users\\Enrico.bisordi\\OneDrive - Rio Bravo\\Desktop\\VAR\\ipca_exp.csv")
petroleo = ip.timeseries('EIA366_PBRENT366')
IBOV = ip.timeseries('ANBIMA12_IBVSP12')
IR = ip.timeseries('SRF12_IR12')

In [19]:
ipca_esperado

,date,median
0,2000-01-01,0.580
1,2000-02-01,0.500
2,2000-03-01,0.700
3,2000-04-01,0.700
4,2000-05-01,0.510
...,...,...
290,2024-03-01,0.500
291,2024-04-01,0.300
292,2024-05-01,0.300
293,2024-06-01,0.240


In [33]:
list_sgs = ['ipca','cambio', 'pib', 'ipgm','igpdi','M1', 'empregos', 'bp', 'divida']
list2 = ['petroleo', 'IBOV', 'IR']

In [32]:
list_sgs = ['ipca','cambio', 'pib', 'ipgm','igpdi','M1', 'empregos', 'bp', 'divida']
list2 = ['petroleo', 'IBOV', 'IR']
colunas = ['CODE', 'RAW DATE', 'DAY', 'MONTH', 'YEAR']

petroleo.drop(columns = colunas, inplace = True)
IBOV.drop(columns = colunas, inplace = True)
IR.drop(columns = colunas, inplace = True)

KeyError: "['CODE', 'RAW DATE', 'DAY', 'MONTH', 'YEAR'] not found in axis"

In [24]:
dict_rename = {'date':'Date'}
ipca_esperado.rename(columns=dict_rename, inplace=True)
ipca_esperado.set_index('Date', inplace = True)

In [26]:
IBOV = IBOV.rename_axis('Date')
IR = IR.rename_axis('Date')
petroleo = petroleo.rename_axis('Date')

In [29]:
cambio

,Câmbio
Date,
1984-11-28,2828.0000
1984-11-29,2828.0000
1984-11-30,2881.0000
1984-12-03,2881.0000
1984-12-04,2881.0000
...,...
2024-07-24,5.6346
2024-07-25,5.6402
2024-07-26,5.6465


In [30]:
def group_by_year_month(df):
    df['Year'] = df.index.year
    df['Month'] = df.index.month
    df_grouped = df.groupby(['Year', 'Month']).mean()
    return df_grouped

In [34]:
grouped_dfs = [group_by_year_month(list_sgs[key]) for key in list_sgs]
grouped_dfs.append(group_by_year_month(ipca_esperado))
grouped_dfs.extend([group_by_year_month(list2[key]) for key in list2])

TypeError: list indices must be integers or slices, not str